In [2]:
# 1. Import Libraries (Minimal)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("Libraries loaded! ✅")

Libraries loaded! ✅


In [3]:
# 2. Load Dataset (Breast Cancer)
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()  # Yeh sklearn ka built-in dataset hai

# DataFrame banate hain (Excel jaisa table)
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target  # 0 = Cancer, 1 = No Cancer

print("Dataset loaded! ✅")
print(f"Total Rows: {len(df)} | Features: {len(data.feature_names)}")
print("First 5 Rows:")
df.head()  # Pehle 5 rows dikhao

Dataset loaded! ✅
Total Rows: 569 | Features: 30
First 5 Rows:


,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [4]:
# 3. Train-Test Split (80-20)
X = df.drop('target', axis=1)  # Features (sab columns except 'target')
y = df['target']               # Labels (0 or 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Data split done! ✅")
print(f"Train: {X_train.shape} | Test: {X_test.shape}")

Data split done! ✅
Train: (455, 30) | Test: (114, 30)


In [5]:
# 4. Feature Scaling + Selection
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  # Train data normalize
X_test_scaled = scaler.transform(X_test)       # Test data normalize

# Top 10 Features select karo (ANOVA F-test se)
selector = SelectKBest(score_func=f_classif, k=10)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

# Selected features ke names print karo
selected_features = X.columns[selector.get_support()]
print("\nTop 10 Features Selected: ✅")
print(selected_features)



Top 10 Features Selected: ✅
Index(['mean radius', 'mean perimeter', 'mean area', 'mean concavity',
       'mean concave points', 'worst radius', 'worst perimeter', 'worst area',
       'worst concavity', 'worst concave points'],
      dtype='object')


In [6]:
# 5. Model Training (Without Feature Selection)
# Logistic Regression
lr = LogisticRegression(max_iter=1000)  # Increase iterations for convergence
lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

# Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_test_scaled)

# Accuracy Calculate karo
lr_acc = accuracy_score(y_test, y_pred_lr)
rf_acc = accuracy_score(y_test, y_pred_rf)

print("Models trained with ALL features! ✅")
print(f"Logistic Regression Accuracy: {lr_acc:.2f}")
print(f"Random Forest Accuracy: {rf_acc:.2f}")

Models trained with ALL features! ✅
Logistic Regression Accuracy: 0.97
Random Forest Accuracy: 0.96


In [7]:
# 6. Model Training (With Feature Selection)
# Logistic Regression (Selected Features)
lr_selected = LogisticRegression(max_iter=1000)
lr_selected.fit(X_train_selected, y_train)
y_pred_lr_selected = lr_selected.predict(X_test_selected)

# Random Forest (Selected Features)
rf_selected = RandomForestClassifier(random_state=42)
rf_selected.fit(X_train_selected, y_train)
y_pred_rf_selected = rf_selected.predict(X_test_selected)

# Accuracy Calculate karo
lr_selected_acc = accuracy_score(y_test, y_pred_lr_selected)
rf_selected_acc = accuracy_score(y_test, y_pred_rf_selected)

print("\nModels trained with SELECTED features! ✅")
print(f"Logistic Regression Accuracy (10 Features): {lr_selected_acc:.2f}")
print(f"Random Forest Accuracy (10 Features): {rf_selected_acc:.2f}")

# Comparison Table
comparison = pd.DataFrame({
    'Model': ['Logistic Regression', 'Random Forest'],
    'All Features Accuracy': [lr_acc, rf_acc],
    'Selected Features Accuracy': [lr_selected_acc, rf_selected_acc],
    'Difference': [lr_acc - lr_selected_acc, rf_acc - rf_selected_acc]
})
print("\nComparison:")
comparison


Models trained with SELECTED features! ✅
Logistic Regression Accuracy (10 Features): 0.97
Random Forest Accuracy (10 Features): 0.96

Comparison:


,Model,All Features Accuracy,Selected Features Accuracy,Difference
0,Logistic Regression,0.973684,0.973684,0.000000
1,Random Forest,0.964912,0.956140,0.008772


In [11]:
# Ensure these variables exist
from sklearn.metrics import accuracy_score
y_pred_lr_selected = lr_selected.predict(X_test_selected)
y_pred_rf_selected = rf_selected.predict(X_test_selected)

In [12]:
import joblib

# Save models & dependencies
joblib.dump(lr_selected, 'logistic_regression_selected.pkl')  # Selected features model
joblib.dump(scaler, 'scaler.pkl')            # Feature scaler
joblib.dump(selector, 'selector.pkl')        # Feature selector

print("Models saved! ✅")

Models saved! ✅
